In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

import numpy as np
import matplotlib.pyplot as plt


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

170498071/170498071 [==============================] - 8s 0us/step


In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [4]:
model_bn_drop = tf.keras.models.Sequential()
model_bn_drop.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu', input_shape = (32, 32, 3)))
model_bn_drop.add(tf.keras.layers.BatchNormalization())
model_bn_drop.add(tf.keras.layers.Dropout(0.15)) 

model_bn_drop.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu'))
model_bn_drop.add(tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = 2, padding = 'same'))
model_bn_drop.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu'))
model_bn_drop.add(tf.keras.layers.BatchNormalization())
model_bn_drop.add(tf.keras.layers.Dropout(0.15)) 

model_bn_drop.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu'))
model_bn_drop.add(tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = 2, padding = 'same'))
model_bn_drop.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu'))
model_bn_drop.add(tf.keras.layers.BatchNormalization())
model_bn_drop.add(tf.keras.layers.Dropout(0.15)) 

model_bn_drop.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu'))
model_bn_drop.add(tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = 2, padding = 'same'))
model_bn_drop.add(tf.keras.layers.Flatten())
model_bn_drop.add(tf.keras.layers.BatchNormalization())
model_bn_drop.add(tf.keras.layers.Dropout(0.15)) 

model_bn_drop.add(tf.keras.layers.Dense(256, activation = 'relu'))
model_bn_drop.add(tf.keras.layers.BatchNormalization())
model_bn_drop.add(tf.keras.layers.Dropout(0.15)) 

model_bn_drop.add(tf.keras.layers.Dense(10, activation = 'softmax'))

model_bn_drop.compile(optimizer = Adam(1e-4),
             loss = 'categorical_crossentropy',
             metrics = ['acc'])

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

```
datagen = ImageDataGenerator(
        rotation_range ,        #이미지 회전 범위 (degrees)
        width_shift_range = 0.2,    #그림을 수평 또는 수직으로 랜덤하게 평행 이동시키는 범위 (원본 가로, 세로 길이에 대한 비율 값)
        height_shift_range = 0.2,    
        rescale = 1./255,             #0~255 -> 0~1
        shear_range = 0.2,           #임의 전단 변환 (shearing transformation) 범위
        zoom_range = 0.2,           #임의 확대/축소 범위
        horizontal_flip = True,       #True로 설정할 경우, 50% 확률로 이미지를 수평으로 뒤집음. 
        원본 이미지에 수평 비대칭성이 없을 때 효과적. 즉, 뒤집어도 자연스러울 때 사용하면 좋음.
        fill_mode = `nearest`)        #이미지를 회전, 이동하거나 축소할 때 생기는 공간을 채우는 방식

```

In [7]:
datagen = ImageDataGenerator(
    featurewise_center = True,            # mean = 0.0
    featurewise_std_normalization= True,  # std = 1.0
    rotation_range=10,    
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    validation_split=0.2)

datagen.fit(x_train)
print("datagen.mean = ", datagen.mean)
print("datagen.mean = ",  datagen.std)

train_generator= datagen.flow(x=x_train, y=y_train, batch_size=400, subset='training')
valid_generator= datagen.flow(x=x_train, y=y_train, batch_size=400, subset='validation')

train_steps= int(np.ceil(train_generator.n/train_generator.batch_size))
valid_steps= int(np.ceil(valid_generator.n/valid_generator.batch_size))
print("train_steps=", train_steps)
print("valid_steps=", valid_steps)

datagen.mean =  [[[125.3069  122.95015 113.866  ]]]
datagen.mean =  [[[62.993256 62.08861  66.705   ]]]
train_steps= 100
valid_steps= 25


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
es = EarlyStopping(patience = 10)
lr = ReduceLROnPlateau(factor = 0.25, patience= 5)

In [8]:
ret = model_bn_drop.fit(train_generator, epochs=100,
                validation_data=valid_generator,
                steps_per_epoch= train_steps,
                validation_steps=valid_steps,
                verbose=1,
                callbacks = [es,lr])

Epoch 1/100
100/100 - 39s - loss: 2.2365 - acc: 0.2848 - val_loss: 2.9271 - val_acc: 0.1030 - 39s/epoch - 391ms/step
Epoch 2/100
100/100 - 29s - loss: 1.8155 - acc: 0.3803 - val_loss: 3.3041 - val_acc: 0.1061 - 29s/epoch - 287ms/step
Epoch 3/100
100/100 - 29s - loss: 1.6467 - acc: 0.4268 - val_loss: 3.2157 - val_acc: 0.1286 - 29s/epoch - 294ms/step
Epoch 4/100
100/100 - 29s - loss: 1.5515 - acc: 0.4543 - val_loss: 2.3682 - val_acc: 0.2560 - 29s/epoch - 293ms/step
Epoch 5/100
100/100 - 31s - loss: 1.4904 - acc: 0.4761 - val_loss: 1.9041 - val_acc: 0.3491 - 31s/epoch - 309ms/step
Epoch 6/100
100/100 - 29s - loss: 1.4351 - acc: 0.4913 - val_loss: 1.6888 - val_acc: 0.4069 - 29s/epoch - 289ms/step
Epoch 7/100
100/100 - 29s - loss: 1.3890 - acc: 0.5070 - val_loss: 1.5495 - val_acc: 0.4577 - 29s/epoch - 292ms/step
Epoch 8/100
100/100 - 28s - loss: 1.3486 - acc: 0.5222 - val_loss: 1.4855 - val_acc: 0.4801 - 28s/epoch - 282ms/step
Epoch 9/100
100/100 - 29s - loss: 1.3057 - acc: 0.5372 - val_los